In [4]:
import credentials
import settings

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob

class MyStreamerListener(tweepy.StreamListener):
    def on_status(self, status):
        if status.retweeted:
            return False
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)
        sentiment = TextBlob(status.text)
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count = status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude= status.coordinates['coordinates']
            latitude = status.coordinates['coordinates']
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        print(status.text)
        print("Long: {}, Lat: {}". format(longitude, latitude))
        
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location,user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql,val)
            mydb.commit()
            mycursor.close()


In [6]:
def clean_tweet(self, tweet): 

    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
                                
def deEmojify(text):
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [7]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="gaurav",
    database="TwitterDB",
    charset = 'utf8'
)

if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()


In [8]:
auth  = tweepy.OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [11]:
t = True
while(t == True):
    myStreamerListener = MyStreamerListener()
    myStream = tweepy.Stream(auth = api.auth, listener = myStreamerListener)
    myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
mydb.close()

Starbucks sees US reopening, but not business as usual https://t.co/pKG85sDS7G via @financialtimes
Long: None, Lat: None
RT @DearVaughn: took her to starbucks cause i forgot her name
Long: None, Lat: None
RT @DearVaughn: took her to starbucks cause i forgot her name
Long: None, Lat: None
RT @Reuters: Starbucks expects China recovery by September as coronavirus seen easing https://t.co/bBopRtLVeP https://t.co/faI3qOBh7c
Long: None, Lat: None
RT @valjbishop61: Be aware Patriots.
Long: None, Lat: None
RT @DearVaughn: took her to starbucks cause i forgot her name
Long: None, Lat: None
RT @DearVaughn: took her to starbucks cause i forgot her name
Long: None, Lat: None
u learned her name but not how to spell it
Long: None, Lat: None
RT @DearVaughn: took her to starbucks cause i forgot her name
Long: None, Lat: None
RT @DearVaughn: took her to starbucks cause i forgot her name
Long: None, Lat: None
RT @Reuters: Starbucks expects China recovery by September as coronavirus seen easing https://t

MySQLInterfaceError: Python type list cannot be converted